## OpenAQ API

- From the OpenAQ API, download a list of reference grade location_ids using the 'us' country code

import requests
import json

# get reference grade sensors in the United States
url = "https://api.openaq.org/v2/locations?limit=4000&page=1&offset=0&sort=desc&country=US&order_by=lastUpdated&sensor_type=reference%20grade&dump_raw=false"

headers = {
    'accept': 'application/json'
}

api_data = requests.get(url, headers=headers).json()

# get all location ids
location_ids = []
for i in api_data["results"]:
    if i['isMobile'] == False:
        location_ids.append(i["id"])

## Downloading the OpenAQ data from AWS

import boto3
from botocore import UNSIGNED
from botocore.config import Config

import pandas as pd
import csv
import gzip
import os

import concurrent.futures
from tqdm import tqdm

s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

#----- Functions -----
def download_and_process_files(key):
    # download object from S3
    # print(f'Downloading {key}')
    obj = s3.get_object(Bucket=bucket_name, Key=key)

    # unzip and store in dataframe
    try:
        with gzip.open(obj['Body'], 'rb') as f:
            df = pd.read_csv(f)
        return df
    except Exception as e:
        print(f'Error in Gzip process: {e}')
# --------------------

# OpenAQ bucket name and base path
bucket_name = 'openaq-data-archive'
base_s3_path = 'records/csv.gz'

# location of CSV with location IDs
location_ids_csv = './location_ids.csv'

# parquet output path
parquet_path = 'G:/_data'

# number of workers used to download from S3 in parallel
num_workers = 200

location_ids = []

with open(location_ids_csv, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        location_ids.append(row[0])

for i in range(len(location_ids)):

    location_id = location_ids[i]

    # check for existing local file
    if os.path.exists(f'{parquet_path}/{location_id}.parquet'):
        print(f'Location {location_id} [{i + 1}/{len(location_ids)}] : Already downloaded')
        continue

    # set S3 path
    s3_path = f'{base_s3_path}/locationid={location_id}'

    # init pagination variables
    continuation_token = None
    s3_files = []

    # paginate through the objects
    while True:
        if continuation_token:
            response = s3.list_objects_v2(Bucket=bucket_name, Prefix=s3_path, ContinuationToken=continuation_token)
        else:
            response = s3.list_objects_v2(Bucket=bucket_name, Prefix=s3_path)

        s3_files.extend([obj['Key'] for obj in response.get('Contents', [])])
        
        if 'NextContinuationToken' in response:
            continuation_token = response['NextContinuationToken']
        else:
            break

    # finished grabbing list of files for this location, print number of files
    print(f'Location {location_id} [{i + 1}/{len(location_ids)}] : {len(s3_files)} files')

    # download objects in parallel, using tqdm to track progress
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        dataframes = list(tqdm(executor.map(download_and_process_files, s3_files), total=len(s3_files)))

    # write Parquet file
    try:
        parquet_file = f'{parquet_path}/{location_id}.parquet'
        print(f'Writing Parquet file: {parquet_file}')
        pd.concat(dataframes, ignore_index=True).to_parquet(parquet_file, engine='pyarrow', index=False)
    except Exception as e:
        print(f'Error writing Parquet file: {e}')

## Local


In [1]:
import numpy as np
import pandas as pd
import os
import sys

import matplotlib.pyplot as plt

import pyspark.sql.functions as F
import pyspark.sql.types as T

os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.cores", "6") \
    .getOrCreate()

## Load Parquet files

In [2]:
# load parquet files
parquet_path = 'G:\\_data'
df = spark.read.parquet(parquet_path)

In [3]:
# count rows
df.count()

165618329

In [4]:
# show schema
df.printSchema()

root
 |-- location_id: long (nullable = true)
 |-- sensors_id: long (nullable = true)
 |-- location: string (nullable = true)
 |-- datetime: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- parameter: string (nullable = true)
 |-- units: string (nullable = true)
 |-- value: double (nullable = true)
 |-- lat.1: double (nullable = true)
 |-- lon.1: double (nullable = true)



## Preprocessing Data

### Drop unused columns

Some source files have errant lat.1 and lon.1 columns. They are all NULL for all rows, and can be safely dropped.

In [5]:
# drop lat.1 and lon.1
df = df.drop('lat.1', 'lon.1')

### Drop NULLs

In [6]:
# remove nulls from chosen columns
df = df.na.drop(subset=['location_id', 'location', 'value', 'parameter'])

### Drop rows with Value 0 or less

In [7]:
df.filter(F.col("value") <= 0).count()

24453897

In [8]:
# keep only values > 0
df = df.filter(F.col("value") > 0)

### Filter out Latitude = Longitude

Some rows are for locations that have the exact same longitude and latitude, which shouldn't be.

The names of these places are often for locations not in the US. We'll filter them out.  

In [9]:
# find rows where latitude and longitude are the exact same
count_lat_long_same = df.filter(F.col('lat') == F.col('lon')).count()
print(f'Number of rows where lat and long are the same: {count_lat_long_same}')

# example
df.filter(F.col('lat') == F.col('lon')).select('location', 'lat', 'lon').limit(1).show()

# filter out rows where lat and lon are equal
df = df.filter(F.col('lat') != F.col('lon'))

Number of rows where lat and long are the same: 7353993
+--------------+---------+---------+
|      location|      lat|      lon|
+--------------+---------+---------+
|EH1F022-237357|4.3268808|4.3268808|
+--------------+---------+---------+



### Pollutants and their Units

- we'll examine the pollutants and their units, and remove some we aren't interested in.
- we'll also align things like the PM25's µg/m³ units:  µ (micro sign, Unicode: U+00B5) and μ (Greek letter mu, Unicode: U+03BC) are different characters. 

In [10]:
# get unique parameters and their units
df_pollutants = df.groupBy('parameter', 'units').count().sort('count')
df_pollutants.show(100)

+----------------+-------------+--------+
|       parameter|        units|   count|
+----------------+-------------+--------+
|            pm25|        μg/m³|   19877|
|             no2|          ppb|   82077|
|              bc|        µg/m³|  151549|
|relativehumidity|            %|  170299|
|             voc|          iaq|  271634|
|             nox|          ppm|  435455|
|              no|          ppm|  475816|
|           um100|particles/cm³|  545981|
|             nox|        µg/m³| 1199185|
|           um050|particles/cm³| 1267610|
|              co|        µg/m³| 1317566|
|              no|        µg/m³| 1574084|
|     temperature|            c| 2322397|
|             so2|        µg/m³| 2767543|
|             so2|          ppm| 3047723|
|              co|          ppm| 3137804|
|           um025|particles/cm³| 3648657|
|     temperature|            f| 3876648|
|             pm1|        µg/m³| 5220020|
|           um010|particles/cm³| 5548547|
|             no2|          ppm| 5

In [11]:
# drop some variants of pm25 we aren't interested in
df = df.filter((F.col('parameter') != 'pm25') | (F.col('units') != 'ppm'))

# any units of 'ppb' have scant rows
df = df.filter(F.col('units') != 'ppb')

# any rows with units of 'particles/cm3' we aren't interested in
df = df.filter(F.col('units') != 'particles/cm³')

# filter out some other pollutants we're not interested in
df = df.filter(~F.col('parameter').isin(['pm25-old', 'ch4', 'bc', 'nox', 'voc', 'no', 'co2', 'pm4']))

# align ug/m3 units
# NOTE: the string values below for ug/m3 should not be touched, retyped, etc. The micro/mu are specific characters that look the same, but are different
df = df.withColumn('units', F.when(F.col('units') == 'μg/m³', 'µg/m³').otherwise(F.col('units')))

#### Convert Temperatures in C to F

In [12]:
# convert temperatures with units C to F
df = df.withColumn('value',
    F.when((F.col('parameter') == 'temperature') & (F.col('units') == 'c'),
        (F.col('value') * 1.8) + 32)
        .otherwise(F.col('value')
    )
)

# change all units to F when parameter is 'temperature'
df = df.withColumn('units', F.when(F.col('parameter') == 'temperature', 'f').otherwise(F.col('units')))

#### Change Pressure Units

In [13]:
# 1 hectopascal equals 1 millibar, so we'll just change the units to mb
df = df.withColumn('units', F.when(F.col('units') == 'hpa', 'mb').otherwise(F.col('units')))

#### Convert units for some pollutants

**NOTE**: skipping this for now. Conversions vary depending on factors, and for now we'll concetrate on rows with the proper units

- for some pollutants, there are measurements in both ug/m3 and ppm
- the AQI calculator (EPA algorithm) expects that pollutants have certain units:
  - CO - ppm *
  - NO2 - ppb *
  - O3 - ppm *
  - PM10 - ug/m3
  - PM25 - ug/m3
  - SO2 - ppb *
- starred units need conversion in this dataset

In [14]:
# TODO: from web research, the converions between ug/m3 and ppb can be quite different depending on temp and pressure, etc. For now, we'll just only use rows with the proper units.

# get a few rows where parameter is 'co' and unit is 'ug/m3'
# z = df.filter((F.col('parameter') == 'co') & (F.col('units') == 'µg/m³')).limit(5)

# NOTE: For now, removing rows with improper units
df = df.filter(~(F.col('parameter').isin(['co', 'no2', 'so2']) & (F.col('units') == 'µg/m³')))

In [15]:
# convert no2 and so2 to ppb
df = df.withColumn('value',
    F.when(
        (F.col('parameter').isin(['so2', 'no2'])) & (F.col('units') == 'ppm'),
        F.col('value') * 1000)
        .otherwise(F.col('value')
    )
)
df = df.withColumn('units',
    F.when(
        (F.col('parameter').isin(['so2', 'no2'])) & (F.col('units') == 'ppm'),
        'ppb')
        .otherwise(F.col('units')
    )
)

#### After this work on Parameters and Units, we'll create and cache the Pollutants for later use

In [16]:
df_pollutants = df.groupBy('parameter', 'units').count().drop('count').sort('parameter', 'units').cache()
df_pollutants.show(100)

+----------------+-----+
|       parameter|units|
+----------------+-----+
|              co|  ppm|
|        humidity|    %|
|             no2|  ppb|
|              o3|  ppm|
|              o3|µg/m³|
|             pm1|µg/m³|
|            pm10|µg/m³|
|            pm25|µg/m³|
|        pressure|   mb|
|relativehumidity|    %|
|             so2|  ppb|
|     temperature|    f|
+----------------+-----+



#### We'll use rough Latitude / Longitude bounding box to limit to US stations

In [17]:
# latitude and longitude bounding box for US
bounding_north = 50
bounding_south = 24
bounding_east = -66
bounding_west = -125

df = df.filter((F.col('lat') >= bounding_south) & (F.col('lat') <= bounding_north) & (F.col('lon') >= bounding_west) & (F.col('lon') <= bounding_east))

#### Daily Averages

- average rows per location, parameter, and day
- i.e. average values for each unique combination of location, parameter, and date

In [18]:
# convert datetime column to yyyy-mm-dd as 'date'
df = df.withColumn('date', F.date_format(F.col('datetime'), 'yyyy-MM-dd'))

# as we're averaging on day, we don't need the precision of the datetime column
df = df.drop('datetime')

# also don't need sensors_id or units column anymore
df = df.drop('sensors_id', 'units')

df = df.groupBy('location_id', 'parameter', 'date').agg(
    F.first('location').alias('location'),
    F.first('lat').alias('lat'),
    F.first('lon').alias('lon'),
    F.avg('value').alias('value')
)

#### Pivot Pollutants

In [19]:
# pivot parameter column
df = df.groupBy('location_id', 'location', 'date', 'lat', 'lon').pivot('parameter').agg(F.avg('value'))

##### Drop rows where PM25 is null

In [20]:
# drop rows where pm25 is null
df = df.na.drop(subset=['pm25'])

### Geocode

We'll Geocode the latitude and longitude values to:

- cull locations that aren't in the US
- use the added categorical geographic information (State, City, etc) in analysis

In [21]:
import reverse_geocoder as rg

def geocode(locations_dict):
    
    # get list of lat, lon tuples from coords dict
    coords = [(row['coords'][0], row['coords'][1]) for row in locations_dict.values()]

    # reverse geocode
    geocode_results = rg.search(coords)

    geocode_results = [{'city': loc['name'], 'state': loc['admin1'], 'country_code': loc['cc']} for loc in geocode_results]
    results = []
    for (key, _), loc in zip(locations_dict.items(), geocode_results):
        results.append((int(key), loc['city'], loc['state'], str.lower(loc['country_code'])))

    return results

# create a UDF with a map(string, string) return type (e.g. {country_code -> 'US'})
geocode_udf = F.udf(geocode, T.MapType(T.StringType(), T.StringType()))

In [22]:
# create dataframe of just distinct locations and their lat, lon
locations = df.groupBy('location_id').agg(F.first('lat').alias('lat'), F.first('lon').alias('lon')).collect()

# setup dictionary, where key is location_id, and value is a tuple of (lat, lon)
locations = {row['location_id']: {'coords': (row['lat'], row['lon'])} for row in locations}

# reverse geocode
locations = geocode(locations)

# create spark dataframe of results
schema = T.StructType([
    T.StructField('location_id', T.LongType(), True),
    T.StructField('city', T.StringType(), True),
    T.StructField('state', T.StringType(), True),
    T.StructField('country_code', T.StringType(), True)
])
geocoded_df = spark.createDataFrame(locations, schema=schema)

# join this dataframe with the original dataframe
df = df.join(geocoded_df, on='location_id', how='left')

Loading formatted geocoded file...


#### Drop rows that aren't in the United States

In [25]:
df = df.filter((F.col('country_code')) == 'us')

### AQI Scores

In [27]:
import aqi

def calculate_aqi(pm10, pm25, co, no2, o3, so2):

    try:
        pollutants = []

        pollutants.append((aqi.POLLUTANT_PM10, pm10)) if pm10 else None
        pollutants.append((aqi.POLLUTANT_PM25, pm25)) if pm25 else None
        pollutants.append((aqi.POLLUTANT_CO_8H, co)) if co else None
        pollutants.append((aqi.POLLUTANT_NO2_1H, no2)) if no2 else None
        pollutants.append((aqi.POLLUTANT_O3_8H, o3)) if o3 else None
        pollutants.append((aqi.POLLUTANT_SO2_1H, so2)) if so2 else None

        return(float(aqi.to_aqi(pollutants)))

    except Exception as e:
        return None
# -------------------------------

# register UDF
aqi_udf = F.udf(calculate_aqi, T.FloatType())

# add AQI column
df = df.withColumn('aqi', aqi_udf(F.col('pm10'), F.col('pm25'), F.col('co'), F.col('no2'), F.col('o3'), F.col('so2')))

In [ ]:
def aqi_category(aqi_score):
    if aqi_score is None:
        return None
    elif aqi_score <= 50:
        return "Good"
    elif aqi_score <= 100:
        return "Moderate"
    elif aqi_score <= 150:
        return "Unhealthy for Sensitive Groups"
    elif aqi_score <= 200:
        return "Unhealthy"
    elif aqi_score <= 300:
        return "Very Unhealthy"
    elif aqi_score > 300:
        return "Hazardous"
    else:
        return None

# register UDF
aqi_category_udf = F.udf(aqi_category, T.StringType())

df = df.withColumn("aqi_category", aqi_category_udf(F.col("aqi")))

## Model - Linear Regression

#### Looking at Average PM2.5 values

In [ ]:
# create a dataframe of average pm25 by date. this is averaged over all locations
df_pm25 = df.groupBy('date').agg(F.avg('pm25').alias('pm25'))
_pm25 = df_pm25.toPandas()

In [ ]:
# convert to datetime
_pm25['date'] = pd.to_datetime(_pm25['date'])

# sort by date
_pm25 = _pm25.sort_values('date')


plt.figure(figsize=(10, 6))
plt.plot(_pm25['date'], _pm25['pm25'], marker='o', linestyle='-')

plt.title('PM2.5 Values in US')
plt.xlabel('Date')
plt.ylabel('Average PM2.5 (µg/m³)')
plt.grid(True)

plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

#### Split data into  training and tests sets, create model

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

df_model = df

# feature engineering: extract date parts
df_model = df_model.withColumn("year", F.year("date"))
df_model = df_model.withColumn("month", F.month("date"))
df_model = df_model.withColumn("day", F.dayofmonth("date"))
df_model = df_model.withColumn("day_of_week", F.dayofweek("date"))

# creating feature vectors
assembler = VectorAssembler(inputCols=["year", "month", "day", "day_of_week"], outputCol="features")
df_model = assembler.transform(df_model)

# split into test and training sets based on 80/20
train_df, test_df = df_model.randomSplit([0.8, 0.2])

# split into test and training sets based on date
# train_df = df_model.filter(F.col('date') < '2023-01-01')
# test_df = df_model.filter(F.col('date') >= '2023-01-01')

# create a linear regression model
lr_model = LinearRegression(featuresCol='features', labelCol='pm25').fit(train_df)

# print the coefficients and intercept for linear regression
print(f"Coefficients: {lr_model.coefficients}")
print(f"Intercept: {lr_model.intercept}")

#### Evaluate the model

In [ ]:
df_model = df

# converting date column to timestamp 
df_model = df.withColumn('timestamp', F.unix_timestamp(F.col('date'), 'yyyy-MM-dd'))

# creating feature vectors
assembler = VectorAssembler(inputCols=["timestamp"], outputCol="features")
df_model = assembler.transform(df_model)

# split into test and training sets based on date
train_df = df_model.filter(F.col('date') < '2023-01-01')
test_df = df_model.filter(F.col('date') >= '2023-01-01')

# create a linear regression model
lr_model = LinearRegression(featuresCol='features', labelCol='pm25').fit(train_df)

# print the coefficients and intercept for linear regression
print(f"Coefficients: {lr_model.coefficients}")
print(f"Intercept: {lr_model.intercept}")

In [ ]:
# predictions on training data
train_predictions = lr_model.transform(train_df)

# predictions on testing data
test_predictions = lr_model.transform(test_df)

evaluator = RegressionEvaluator(labelCol="pm25", predictionCol="prediction", metricName="rmse")

# RMSE for training
train_rmse = evaluator.evaluate(train_predictions)

# RMSE for testing
test_rmse = evaluator.evaluate(test_predictions)

print("Training RMSE:", train_rmse)
print("Testing RMSE:", test_rmse)

## Model fitting

Our model fits in the underfitting side of the fitting graph. Our linear regression model is on the more simple side, with only one parameter reading vs. timestamp as the input of trying to predict future values. We started off with pm values across many stations, but averaged these across the entire US and across each day as an input to our model, which simplified the model. Unforunately, the values in our dataset only go a couple of years back. Because it is more simple due to these reasons, the model is prone to underfitting and not being as robust as if we had added more time points.


## Next Models

The next models we are thinking of will utilize more of the parameters we have in our dataset. We can apply k-means clustering to obtain clusters of places with similar levels/patterns of pollution based on features. This would be interesting to investigate as we could look into areas of best and worst pollution in the US. We could also run PCA on these parameter features for dimensionality reduction on our dataset as we have many parameters of data to pool from.

## Conclusions